In [20]:
import gc
import multiprocessing
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score
from sklearn.utils import resample

In [21]:
df = pd.read_csv("../curated-data/ModelData.csv")
df.head(5)

,NASA,TA,EXT,AGR,CS,NT,OP,AV,EM,Task,...,BR,NP,SR,AP,AR,DWH,DWR,T,DS,Rank
0,24,38,2,8,10,3,8,16,17,26,...,2,2,1,1,4,2,70,1,1,2
1,17,35,6,9,7,4,8,25,14,31,...,2,2,1,1,5,1,70,1,1,1
2,16,54,3,7,3,9,6,23,24,16,...,1,2,0,2,2,2,60,1,2,0
3,18,28,6,4,10,3,8,13,12,35,...,1,1,1,2,5,1,60,2,1,2
4,15,42,7,8,10,8,10,22,16,30,...,2,2,0,2,4,2,80,2,2,1


In [22]:
df = df[df.Rank == 2]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 359
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NASA    179 non-null    int64  
 1   TA      179 non-null    int64  
 2   EXT     179 non-null    int64  
 3   AGR     179 non-null    int64  
 4   CS      179 non-null    int64  
 5   NT      179 non-null    int64  
 6   OP      179 non-null    int64  
 7   AV      179 non-null    int64  
 8   EM      179 non-null    int64  
 9   Task    179 non-null    int64  
 10  H       179 non-null    float64
 11  RS      179 non-null    int64  
 12  WH      179 non-null    int64  
 13  TWR     179 non-null    int64  
 14  BR      179 non-null    int64  
 15  NP      179 non-null    int64  
 16  SR      179 non-null    int64  
 17  AP      179 non-null    int64  
 18  AR      179 non-null    int64  
 19  DWH     179 non-null    int64  
 20  DWR     179 non-null    int64  
 21  T       179 non-null    int64  
 22  DS

In [23]:
df

,NASA,TA,EXT,AGR,CS,NT,OP,AV,EM,Task,...,BR,NP,SR,AP,AR,DWH,DWR,T,DS,Rank
0,24,38,2,8,10,3,8,16,17,26,...,2,2,1,1,4,2,70,1,1,2
3,18,28,6,4,10,3,8,13,12,35,...,1,1,1,2,5,1,60,2,1,2
5,20,34,5,6,10,5,5,15,18,28,...,2,1,1,2,2,1,50,2,1,2
8,19,22,4,7,10,2,9,14,10,24,...,2,1,0,2,5,1,60,1,1,2
10,16,45,2,10,6,4,7,25,17,29,...,1,2,0,2,5,2,80,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,19,32,2,9,10,6,8,20,17,31,...,1,2,0,2,3,2,70,1,2,2
351,27,37,7,10,10,4,7,21,17,30,...,2,3,0,2,5,2,80,2,1,2
353,18,55,8,7,8,6,10,16,23,28,...,2,2,1,1,3,1,50,2,1,2
357,14,30,4,8,7,6,10,13,12,27,...,2,2,0,1,5,2,80,1,2,2


In [24]:
from sklearn.model_selection import train_test_split
Y = df.SR
X = df.drop('SR', axis=1)

# setting up testing and training sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=27)

In [25]:
X = pd.concat([X_train, Y_train], axis=1)
X.info()
X.SR.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 285 to 34
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NASA    143 non-null    int64  
 1   TA      143 non-null    int64  
 2   EXT     143 non-null    int64  
 3   AGR     143 non-null    int64  
 4   CS      143 non-null    int64  
 5   NT      143 non-null    int64  
 6   OP      143 non-null    int64  
 7   AV      143 non-null    int64  
 8   EM      143 non-null    int64  
 9   Task    143 non-null    int64  
 10  H       143 non-null    float64
 11  RS      143 non-null    int64  
 12  WH      143 non-null    int64  
 13  TWR     143 non-null    int64  
 14  BR      143 non-null    int64  
 15  NP      143 non-null    int64  
 16  AP      143 non-null    int64  
 17  AR      143 non-null    int64  
 18  DWH     143 non-null    int64  
 19  DWR     143 non-null    int64  
 20  T       143 non-null    int64  
 21  DS      143 non-null    int64  
 22  R

0    85
1    58
Name: SR, dtype: int64

In [26]:
Success = X[X.SR==1]
Not_Success = X[X.SR==0]

# upsample minority
success_upsampled = resample(Success,
                          replace=True, # sample with replacement
                          n_samples=len(Not_Success), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([Not_Success, success_upsampled])

# check new class counts
upsampled.SR.value_counts()

1    85
0    85
Name: SR, dtype: int64

In [27]:
%%time
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, Y_train)

Wall time: 158 ms


RandomForestClassifier()

In [28]:
y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)
pd.DataFrame(confusion_matrix(Y_test, y_pred))

,0,1
0,19,4
1,9,4


In [29]:
print("Accuracy: ", accuracy_score(Y_test, y_pred))
print("Precision: ", precision_score(Y_test, y_pred))
print("Recall: ", recall_score(Y_test, y_pred))
print("F1: ", f1_score(Y_test, y_pred))
print("Auc Score: ", roc_auc_score(Y_test, y_score[:,1]))

Accuracy:  0.6388888888888888
Precision:  0.5
Recall:  0.3076923076923077
F1:  0.380952380952381
Auc Score:  0.5819397993311037
